In [ ]:
import os
from google.colab import drive
MOUNTPOINT = '/content/gdrive'
DATADIR = os.path.join(MOUNTPOINT, 'My Drive', 'Data')
drive.mount(MOUNTPOINT, force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install -U sentence-transformers
!pip install clusteval
!pip install bnpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 5.5 MB 37.0 MB/s 
     |████████████████████████████████| 1.3 MB 42.0 MB/s 
     |████████████████████████████████| 182 kB 52.9 MB/s 
     |████████████████████████████████| 7.6 MB 44.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c1490a12aab5ee58f67c7f61c13bfd0f2acb04eb7e647b67296964be45ca33cd
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=8a8b9caeebbbe2eb81c784b9082e0068bb907a93b3c5cc9db260c8e37e990b2b
  Stored in directory: /root/.

In [ ]:
import pandas as pd
import numpy as np
import glob

import xml.etree.ElementTree as ET
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
# import hdbscan

import tensorflow as tf
from tensorflow import keras
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM,Bidirectional, Dropout,Dense
from keras.models import Sequential
import keras.backend as K

from sklearn.metrics import cohen_kappa_score, mean_squared_error , f1_score, accuracy_score
from scipy.stats import pearsonr
import scipy
from statistics import stdev
import bnpy, os
import math
from sklearn.metrics import confusion_matrix
import seaborn as sns
import random
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Choose data
wits = pd.read_csv("/content/gdrive/MyDrive/Data/wits/wits_short.csv")
unique = wits["Question"].unique()
modelbert = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
modelbert.max_seq_length = 512


In [ ]:
modelbert.max_seq_length

512

In [ ]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re
def clean_txt(text):
  text = re.sub("'","",text)
  text = re.sub("(\\W)+"," ",text)
  text = text.lower()
  return text

In [ ]:
from torch import dropout
ck = []
pr = []
er = []
acc = []
f1 = []
conf_matrices = []

ck1 = []
pr1 = []
er1 = []
acc1 = []
f11 = []
conf_matrices1 = []

over = []
short_questions = []
long_questions = []

num_marked_scripts = []
num_marked_scripts2 = []
for ques_num in range(0,len(unique)):

  print("====================================",ques_num,"========================================")
  answers = wits[wits['Question']==unique[ques_num]]
  answers["Response"] = answers.Response.apply(clean_txt)
  answers = answers.reset_index()
  maxscore = int(answers['Grade'].max())
  max_resp_len = len(answers["Response"][answers["Response"].str.len().argmax()].split())

  sent = []
  print((answers))

  for i in range(len(answers)):
    sent.append(answers['Response'].iloc[i])

  # BERT Embeddings
  ref_text_embeddings = modelbert.encode(sent, batch_size = 20, show_progress_bar = True)
  
  ref_text_embeddings = np.nan_to_num(ref_text_embeddings)

  # GMM
  # num_clusters = 1
  # # Define kmeans model
  # clustering_model = BayesianGaussianMixture(n_components=num_clusters,warm_start=True)
  # # Fit the embedding with kmeans clustering.
  # clustering_model.fit(ref_text_embeddings)
  # # Get the cluster id assigned to each news headline.
  # cluster_assignment = clustering_model.predict(ref_text_embeddings)
  
  
  # npBGMM
  dataset = bnpy.data.XData(ref_text_embeddings)
  full_trained_model, full_info_dict = bnpy.run(dataset,'DPMixtureModel',"Gauss","memoVB",output_path='/content/results2/',nLap=20, nBatch=1,ECovMat='eye',sF=0.0000001,moves='birth,merge,shuffle',initname = "bregmankmeans",K=30,b_Kfresh=4,b_startLap=2,m_startLap=2)#
  LP = full_trained_model.obsModel.calc_local_params(dataset)['E_log_soft_ev']
  cluster_assignment = LP.argmax(axis=1)
  num_clusters  = np.max(cluster_assignment) +1

  # Get Clusters
  clustered_sentences = [[] for i in range(num_clusters)]
  for sentence_id, cluster_id in enumerate(cluster_assignment):
      clustered_sentences[cluster_id].append(sentence_id)


  ind = []
  outliers = []
  pred_y = np.zeros(len(answers))
  completed = False


  fraction = 0.0001/(num_clusters)

  flag = True 
  counter = 0

  num_samples = round(25/num_clusters)

  while flag:
    counter +=1
    ind = []

    for i in range(num_clusters):
        density = scipy.stats.multivariate_normal(cov=full_trained_model.obsModel.get_covar_mat_for_comp(i), mean=full_trained_model.obsModel.get_mean_for_comp(i),allow_singular=True).logpdf(ref_text_embeddings)
        
        # for GMM
        # density = scipy.stats.multivariate_normal(cov=clustering_model.covariances_[i], mean=clustering_model.means_[i]).logpdf(ref_text_embeddings)
        
        argsorted = np.argsort(density)
        argsorted2 = []
        for x in reversed(argsorted):
          if ( x in clustered_sentences[i]):
            argsorted2.append(x)
        
        # for random sampling from clusters
        # if len(argsorted2) < num_samples:  
        #   ind = ind + argsorted2
        # else:
        #  ind = ind + random.sample(argsorted2, num_samples)

        if len(argsorted2)==0:
          continue
        f = density[argsorted2[0]]
        ind.append(argsorted2[0])
        mark = answers['Grade'].iloc[argsorted2[0]]

        for p in range(1,len(argsorted2)):
          if f - density[argsorted2[p]] >  fraction*np.std(density[argsorted2]) : #0.005:
            f = density[argsorted2[p]]
            ind.append(argsorted2[p])
            mark = answers['Grade'].iloc[argsorted2[p]] #mark input
            pred_y[argsorted2[p]] = mark
          else :
            pred_y[argsorted2[p]] = mark
        
    flag = False
   
  short_questions.append(ques_num)
  num_marked_scripts.append(len(ind))
      
  ctrain = []
  cy = []
  senttrain = []
  print(len(ind))
  one = OneHotEncoder()

  grades = pd.get_dummies(answers['Grade']).to_numpy()  

  for i in ind:
    senttrain.append(ref_text_embeddings[i])
    cy.append(grades[i])

  valid_ans = []
  testy = []
  for i in range(len(answers)) :
    if not(i in ind):
      valid_ans.append(ref_text_embeddings[i])
      testy.append(grades[i])

  anslen = len(testy)
  print(ref_text_embeddings.shape)
  ctrain = senttrain.copy()
  test = valid_ans.copy()
  max_seq_length = 768
  n_unique_words =10000 # cut texts after this number of words
  maxlen = ref_text_embeddings.shape[1]
  batch_size = 16

  # LSTM Model
  model = Sequential()
  model.add(Bidirectional(LSTM(768, activation='tanh',dropout = 0.5),input_shape=(None,maxlen)))
  model.add(Dense(384, activation='tanh'))
  model.add(Dense(maxscore + 1, activation = "softmax"))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

 
  test = np.array(test)
  ctrain = np.array(ctrain)
  cy = np.array(cy)
  testy = np.array(testy)
  ctrain=ctrain.reshape((-1, 1,maxlen))
  test=test.reshape((-1, 1,maxlen))

  tf.config.run_functions_eagerly(True)
  early_stopping_monitor = EarlyStopping(monitor='loss', patience=2)
  history=model.fit(ctrain, cy,batch_size=batch_size,epochs=100,validation_data=[test,testy])
  print(history.history['loss'])
  print(history.history['accuracy'])

  lstms = model.predict(test)

  lstms = np.argmax(lstms, axis = 1)
  testy = np.argmax(testy, axis = 1)
  

  print(confusion_matrix(testy,lstms))
  print("cohen_kappa_score",cohen_kappa_score(testy,lstms))
  print("pearsonr",pearsonr(testy,lstms))
  print("mean_squared_error",mean_squared_error(testy,lstms))
  print("accuracy_score",accuracy_score(testy,lstms))
  print("f1_score",f1_score(testy,lstms, average = 'weighted'))

  conf_matrices.append(confusion_matrix(testy,lstms))
  ck.append(cohen_kappa_score(lstms, testy))
  pr.append(pearsonr(lstms, testy))
  er.append(mean_squared_error(lstms, testy))
  acc.append(accuracy_score(lstms, testy))
  f1.append(f1_score(lstms, testy, average = 'weighted'))
  long_questions.append(ques_num)
  num_marked_scripts2.append(len(ind))


==================================== 0 ========================================
==================================== 1 ========================================
==================================== 2 ========================================
==================================== 3 ========================================
==================================== 4 ========================================
     index  Unnamed: 0  Unnamed: 0.1  \
0     1267        1267          1301   
1     1268        1268          1302   
2     1269        1269          1303   
3     1270        1270          1304   
4     1271        1271          1305   
..     ...         ...           ...   
320   1587        1587          1621   
321   1588        1588          1622   
322   1589        1589          1623   
323   1590        1590          1624   
324   1591        1591          1625   

                                                    ID  \
0    5ac95126c93bab762f619ae6d9e0457a3ae69e32d351c6...   
1  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Batches:   0%|          | 0/17 [00:00<?, ?it/s]

Dataset Summary:
X Data
  total size: 325 units
  batch size: 325 units
  num. batches: 1
Allocation Model:  DP mixture with K=0. Concentration gamma0= 1.00
Obs. Data  Model:  Gaussian with full covariance.
Obs. Data  Prior:  Gauss-Wishart on mean and covar of each cluster
  E[  mean[k] ] = 
   [0. 0.]  ...
  E[ covar[k] ] = 
  [[1.e-07 0.e+00]
   [0.e+00 1.e-07]] ...
Initialization:
  initname = bregmankmeans
  K = 30 (number of clusters)
  seed = 1607680
  elapsed_time: 13.5 sec
Learn Alg: memoVB | task  1/1 | alg. seed: 1607680 | data order seed: 8541952
task_output_path: /content/results2/1
BIRTH @ lap 1.00: Disabled. Waiting for lap >= 2 (--b_startLap).
MERGE @ lap 1.00: Disabled. Cannot plan merge on first lap. Need valid SS that represent whole dataset.
    1.000/20 after      2 sec. |   1643.1 MiB | K    5 | loss -9.945212935e+00 |  
BIRTH @ lap 2.000 : None attempted. 0 past failures. 0 too small. 5 too busy.
MERGE @ lap 2.00 : 2/2 accepted. Ndiff 147.00. 8 skipped.
    2.000/

/usr/local/lib/python3.7/dist-packages/bnpy/obsmodel/GaussObsModel.py:1101: RuntimeWarning: divide by zero encountered in log
  logdetB = np.log(np.linalg.det(logdetB))
/usr/local/lib/python3.7/dist-packages/bnpy/obsmodel/GaussObsModel.py:641: RuntimeWarning: invalid value encountered in double_scalars
  Gap[j, k] = c[j] + c[k] - cPrior - cjk


...

Dataset Summary:
X Data
  total size: 362 units
  batch size: 362 units
  num. batches: 1
Allocation Model:  DP mixture with K=0. Concentration gamma0= 1.00
Obs. Data  Model:  Gaussian with full covariance.
Obs. Data  Prior:  Gauss-Wishart on mean and covar of each cluster
  E[  mean[k] ] = 
   [0. 0.]  ...
  E[ covar[k] ] = 
  [[1.e-07 0.e+00]
   [0.e+00 1.e-07]] ...
Initialization:
  initname = bregmankmeans
  K = 30 (number of clusters)
  seed = 1607680
  elapsed_time: 14.3 sec
Learn Alg: memoVB | task  1/1 | alg. seed: 1607680 | data order seed: 8541952
task_output_path: /content/results2/1
BIRTH @ lap 1.00: Disabled. Waiting for lap >= 2 (--b_startLap).
MERGE @ lap 1.00: Disabled. Cannot plan merge on first lap. Need valid SS that represent whole dataset.
    1.000/20 after      1 sec. |   3656.4 MiB | K    4 | loss -1.001616192e+01 |  
BIRTH @ lap 2.000 : None attempted. 0 past failures. 0 too small. 4 too busy.
MERGE @ lap 2.00 : 2/2 accepted. Ndiff 92.00. 4 skipped.
    2.000/2

/usr/local/lib/python3.7/dist-packages/bnpy/obsmodel/GaussObsModel.py:1101: RuntimeWarning: divide by zero encountered in log
  logdetB = np.log(np.linalg.det(logdetB))
/usr/local/lib/python3.7/dist-packages/bnpy/obsmodel/GaussObsModel.py:641: RuntimeWarning: invalid value encountered in double_scalars
  Gap[j, k] = c[j] + c[k] - cPrior - cjk


BIRTH @ lap 4.00 : Added 0 states. 0/1 succeeded. 1/1 failed eval phase. 0/1 failed build phase.
    4.000/20 after      9 sec. |   3656.4 MiB | K    1 | loss -1.009610747e+01 | Ndiff    0.000 
MERGE @ lap 5.00: No promising candidates, so no attempts.
BIRTH @ lap 5.000 : None attempted. 1 past failures. 0 too small. 0 too busy.
    5.000/20 after     10 sec. |   3656.4 MiB | K    1 | loss -1.009610747e+01 | Ndiff    0.000 
MERGE @ lap 6.00: No promising candidates, so no attempts.
BIRTH @ lap 6.000 : None attempted. 1 past failures. 0 too small. 0 too busy.
    6.000/20 after     10 sec. |   3656.4 MiB | K    1 | loss -1.009610747e+01 | Ndiff    0.000 
MERGE @ lap 7.00: No promising candidates, so no attempts.
BIRTH @ lap 7.000 : None attempted. 1 past failures. 0 too small. 0 too busy.
    7.000/20 after     11 sec. |   3656.4 MiB | K    1 | loss -1.009610747e+01 | Ndiff    0.000 
MERGE @ lap 8.00: No promising candidates, so no attempts.
BIRTH @ lap 8.000 : None attempted. 1 past fa

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


2/2 [==============================] - 2s 1s/step - loss: 0.6620 - accuracy: 0.5600 - val_loss: 0.4718 - val_accuracy: 0.9110
Epoch 2/100
2/2 [==============================] - 2s 1s/step - loss: 0.4745 - accuracy: 0.8000 - val_loss: 0.2635 - val_accuracy: 1.0000
Epoch 3/100
2/2 [==============================] - 1s 1s/step - loss: 0.2584 - accuracy: 1.0000 - val_loss: 0.0944 - val_accuracy: 1.0000
Epoch 4/100
2/2 [==============================] - 2s 1s/step - loss: 0.0968 - accuracy: 1.0000 - val_loss: 0.0255 - val_accuracy: 1.0000
Epoch 5/100
2/2 [==============================] - 2s 1s/step - loss: 0.0209 - accuracy: 1.0000 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 6/100
2/2 [==============================] - 2s 1s/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 7/100
2/2 [==============================] - 2s 1s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 3.3908e-04 - val_accuracy: 1.0000
Epoch 8/100
2/2 [=========================

In [ ]:
  from pathlib import Path

In [ ]:
data1 = np.array([short_questions,num_marked_scripts,pr1,ck1, acc1, f11, er1, conf_matrices1],dtype=object)
data2 = np.array([long_questions,num_marked_scripts2,pr,ck, acc, f1, er, conf_matrices],dtype=object)
cols = np.array(["question_num","num_marked_samples","pearsons_r","cohens_kappa","accuracy","f1","mse","confusion_matrix"])
filepath1 = Path('/content/gdrive/MyDrive/Data/wits/results/results_small_bn_naive_prediction_random.csv')
filepath2 = Path('/content/gdrive/MyDrive/Data/wits/results/results_small_bn_lstm_random.csv')





In [ ]:
results1 = pd.DataFrame(data=np.transpose(data1),columns = cols)
results1.to_csv(filepath1)

In [ ]:
results2 = pd.DataFrame(data=np.transpose(data2),columns = cols)
results2.to_csv(filepath2)